In [20]:
# This data folder should contain the folder as specified in info.json file
DATA_PATH = "DATA_DIR_TO_SPECIFY"
EXPORT_PATH = "DIR_TO_SAVE_OUTPUT"
# Common path to fastq folder in each data folder
PATH_TO_FASTQ = "PATH_TO_FASTQ_FOLDER"
INFO_FILE = "info.json"

In [2]:
import json, glob, zipfile,os
from sequana import fastqc
from IPython.display import HTML
import pandas as pd

In [3]:
data = json.loads(open(INFO_FILE).read())

sample_names = []
for group in ['A', 'B', 'C', 'D', 'E']:
    sample_names.extend(sorted(data['groups'][group]))


# Reads all data from all fastqc reports

In [21]:
all_results = {}
for key, value in data['runs'].items():
    print("reading {}".format(key))
    path = DATA_PATH + value["path"] + PATH_TO_FASTQ
    filenames  = glob.glob(path+"/*")
    ff = fastqc.FastQC()
    for filename in filenames:
        sample = os.path.basename(filename)
        archive = filename + "/{}_R1_001_fastqc.zip".format(sample)
        ff.read_sample(archive, sample)
    all_results[key] = ff

reading arbo_double_is
reading arbo_double_ms
reading arbo_double_10plex_ms
reading arbo_single_is
reading arbo_single_ms
reading illumina_resv_is
reading illumina_resv_ms
reading illumina_resv_ns
reading meta_is
reading meta_ns
reading twist_cov_is
reading twist_cov_ms
reading twist_cov_ns
reading twist_panviral_is
reading twist_panviral_ns
reading twist_cov_10plex_ms
reading twist_cov_LB_ms


# Extract relevant data and store in single dataframe

In [23]:
X, Y, MQ, GC, TS, DUP, groups, sequencers = [], [], [], [], [], [], [], []
for name in all_results.keys():
    samples = sorted(all_results[name].fastqc_data.keys())
    for sample in samples:
        if sample.startswith("Controle") or sample.startswith('Cneg'):
            continue
        X.append(name)
        
        mq = all_results[name].fastqc_data[sample]['basic_statistics']['mean_quality']
        gc = all_results[name].fastqc_data[sample]['basic_statistics']['%GC']
        ts = all_results[name].fastqc_data[sample]['basic_statistics']['Total Sequences']
        dup = all_results[name].fastqc_data[sample]['basic_statistics']['total_deduplicated_percentage']
        MQ.append(mq)
        GC.append(gc)
        TS.append(ts)
        DUP.append(dup)
        shortname = sample.split("_")[0].split("-")[0]
        Y.append(shortname)
        groups.append(data["samples"][shortname])
        sequencers.append(name.split("_")[-1])


In [24]:
df = pd.DataFrame({"run": X, "sample": Y, "mean_quality": 
                   MQ, "gc": GC, "duplication": DUP, "nreads": TS, 
                   "group": groups, "sequencer": sequencers})

# Data extraction, manipulation

In [26]:
dup = df.pivot_table(index="run", columns="sample")['duplication']
gc = df.pivot_table(index="run", columns="sample")['gc']
nreads = df.pivot_table(index="run", columns="sample")['nreads'] # .fillna(0).astype('int32')
qual = df.pivot_table(index="run", columns="sample")['mean_quality']


In [27]:
mu_qual = qual.mean(skipna=True)
std_qual = qual.std(skipna=True)
mu_gc = gc.mean(skipna=True)
std_gc = gc.std(skipna=True)
mu_nreads = nreads.mean(skipna=True)
std_nreads = nreads.std(skipna=True)
mu_dup = dup.mean(skipna=True)
std_dup = dup.std(skipna=True)


In [28]:
(df.groupby(["run", "sequencer"])['nreads'].sum()/1000000).groupby('sequencer').describe()

,count,mean,std,min,25%,50%,75%,max
sequencer,,,,,,,,
is,6.0,4.618666,1.610151,1.343326,5.092921,5.260690,5.350345,5.430061
ms,7.0,19.884834,1.684560,17.955639,18.767226,20.001158,20.308606,23.085373
ns,4.0,357.885203,197.043626,68.620604,331.728133,425.938559,452.095629,511.043090


In [30]:
summary = pd.concat([mu_qual, std_qual, mu_gc, std_gc, mu_nreads, std_nreads, mu_dup, std_dup], axis=1)
summary.columns = ["quality_mean", "quality_std", "gc_mean", "gc_std", "nreads_mean", "nreads_std", "duplicated_mean", "duplicated_std"]
summary = summary.loc[[str(x) for x in sample_names]]
summary

,quality_mean,quality_std,gc_mean,gc_std,nreads_mean,nreads_std,duplicated_mean,duplicated_std
sample,,,,,,,,
4520,35.515694,1.460262,43.466667,4.983783,6.748163e+06,1.276825e+07,43.118753,26.791542
4660,35.703654,1.585493,44.400000,7.623273,5.040167e+06,9.568181e+06,33.200421,26.278941
4716,35.761965,1.579649,45.066667,5.049281,4.032054e+06,7.902824e+06,49.879232,28.025383
4885,35.724317,1.530337,44.800000,5.212622,4.518900e+06,9.335046e+06,54.246544,28.366015
4653,35.869663,1.551246,44.470588,7.787150,7.066043e+06,1.390534e+07,26.711989,23.163111
4676,35.935952,1.523026,44.647059,5.883676,4.354055e+06,7.399865e+06,40.390964,32.012927
4697,35.851697,1.501272,49.529412,3.104788,3.355449e+06,7.514463e+06,51.226814,21.708977
4707,35.871162,1.514475,45.000000,4.677072,4.709817e+06,1.016958e+07,52.696021,29.715973
4673,35.725900,1.487867,50.000000,4.046604,2.978015e+06,6.334108e+06,44.168179,22.246067


In [ ]:
dup.to_csv(EXPORT_PATH + "Table_QC_dup.csv")
gc.to_csv(EXPORT_PATH + "Table_QC_gc.csv")
qual.to_csv(EXPORT_PATH + "Table_QC_mean_qual.csv")
nreads.to_csv(EXPORT_PATH + "Table_QC_nreads.csv")
summary.to_csv(EXPORT_PATH + "Table_summary.csv")